# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the properties dataset (incl. the auxiliary data or any other data you might have collected); there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* **Important:** Please consider this notebook as an example and not to set specific requirements. Your notebook is likely to look very different. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from dataloader import read_csv
from preprocessing import DataPreprocessor

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load the Data

For this example, we use a simplified version of the dataset with only 2k+ data samples, each with only a subset of features.

In [3]:
trainX, trainY = read_csv('data/train.csv', ylabel='price')

In [4]:
auxSubzone, _ = read_csv('data/auxiliary-data/sg-subzones.csv')

auxInfraDict = {}
Infralist = ['sg-commerical-centres', 'sg-mrt-stations', 'sg-primary-schools', 'sg-secondary-schools', 'sg-shopping-malls']
for ele in Infralist:
    auxInfra, _ = read_csv('data/auxiliary-data/' + ele + '.csv')
    auxInfraDict[ele] = auxInfra

In [5]:
data_preprocessor = DataPreprocessor(auxSubzone, auxInfraDict)
trainX, trainY = data_preprocessor.fit_transform(trainX, trainY)

In [6]:
df_data = pd.concat([trainX, trainY], axis=1)

In [7]:
df_data.head()

,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,total_num_units,lat,...,furnishing_unfurnished,furnishing_fully,floor_level_nan,floor_level_low,floor_level_ground,floor_level_high,floor_level_penthouse,floor_level_mid,floor_level_top,price
0,0.0,0.0,6.744873e+05,0.0,1988.0,3.0,2.0,1115.0,116.0,1.414399,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,514500.0
1,1.0,1.0,6.744873e+05,1.0,1992.0,4.0,2.0,1575.0,407.0,1.372597,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,995400.0
2,2.0,2.0,2.936252e+06,2.0,2022.0,4.0,6.0,3070.0,56.0,1.298773,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8485000.0
3,3.0,3.0,2.936252e+06,2.0,2023.0,3.0,2.0,958.0,638.0,1.312364,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2626000.0
4,4.0,4.0,2.936252e+06,1.0,2026.0,2.0,1.0,732.0,351.0,1.273959,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1764000.0


In [17]:
df_data.iloc[0]

address                          0.000000e+00
property_name                    0.000000e+00
property_type                    6.744873e+05
tenure                           0.000000e+00
built_year                       1.988000e+03
num_beds                         3.000000e+00
num_baths                        2.000000e+00
size_sqft                        1.115000e+03
total_num_units                  1.160000e+02
lat                              1.414399e+00
lng                              1.038372e+02
subzone                          6.719120e+05
planning_area                    1.124002e+06
tenure_duration                  9.900000e+01
is_freehold                      0.000000e+00
nearest_sg-commerical-centres    3.003721e-02
density_sg-commerical-centres    1.352862e-01
nearest_sg-mrt-stations          5.164990e-03
density_sg-mrt-stations          2.585848e+00
nearest_sg-primary-schools       2.478569e-03
density_sg-primary-schools       3.531130e+00
nearest_sg-secondary-schools     1

In [16]:
data_preprocessor.inverse_transform(row).iloc[0]

0    furnishing_unspecified
dtype: object
0    floor_level_nan
dtype: object


address                          sembawang / yishun (d27)
property_name                        866 yishun street 81
property_type                                         hdb
tenure                                                NaN
built_year                                         1988.0
num_beds                                              3.0
num_baths                                             2.0
size_sqft                                          1115.0
total_num_units                                     116.0
lat                                              1.414399
lng                                            103.837196
subzone                                      yishun south
planning_area                                      yishun
tenure_duration                                      99.0
is_freehold                                           0.0
nearest_sg-commerical-centres                    0.030037
density_sg-commerical-centres                    0.135286
nearest_sg-mrt

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [ ]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = 10
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
    
    

        
    # Return the dataset with the k recommendations
    return df_result


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [ ]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df_sample.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [ ]:
k = 3

df_recommendations = get_top_recommendations(row, k=k)

df_recommendations.head(k)